In [ ]:
!pip install pycaret==2.3.10 markupsafe==2.0.1 pyyaml==5.4.1 -qq
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.23.2

     |████████████████████████████████| 320 kB 18.0 MB/s 
     |████████████████████████████████| 636 kB 5.2 MB/s 
     |████████████████████████████████| 262 kB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 45.6 MB/s 
     |████████████████████████████████| 1.3 MB 11.9 MB/s 
     |████████████████████████████████| 6.8 MB 39.7 MB/s 
     |████████████████████████████████| 134 kB 47.4 MB/s 
     |████████████████████████████████| 88 kB 8.4 MB/s 
     |████████████████████████████████| 1.7 MB 48.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 25.9 MB 1.7 MB/s 
     |████████████████████████████████| 2.0 MB 45.8 MB/s 
     |████████████████████████████████| 17.0 MB 36.3 MB/s 
     |████████████████████████████████| 10.4 MB 50.9 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████

In [8]:
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score,roc_auc_score
#from google.colab import drive
import warnings
#drive.mount('/content/drive')

#from pycaret.utils import enable_colab
import jinja2
#enable_colab()
from pycaret.classification import *

# 새 섹션

In [5]:
#1. 내 드라이브에 SW_AI 폴더 생성 
#2. 공유드라이브의 Dataset폴더 바로가기를 자신의 드라이브 SW_AI폴더에 넣음
#3. 실행~_~
train  = pd.read_csv('./drive/MyDrive/SW_AI/Dataset/train.csv')
test  = pd.read_csv('./drive/MyDrive/SW_AI/Dataset/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: './drive/MyDrive/SW_AI/Dataset/train.csv'

In [ ]:
print(train.head())
print(train.shape)
print("Class counting")
print(train['nerdiness'].value_counts())
print(train.isnull().sum())
train_fill_nan = train.fillna(0)
print(train_fill_nan.shape)

   index   Q1   Q2   Q3   Q4   Q5   Q6   Q7   Q8   Q9  Q10  Q11  Q12  Q13  \
0      0  1.0  5.0  5.0  5.0  1.0  4.0  5.0  5.0  1.0  3.0  5.0  5.0  5.0   
1      1  4.0  4.0  4.0  4.0  4.0  5.0  4.0  4.0  3.0  3.0  1.0  4.0  5.0   
2      2  4.0  5.0  5.0  4.0  3.0  5.0  5.0  5.0  4.0  4.0  2.0  5.0  5.0   
3      3  4.0  4.0  4.0  2.0  4.0  3.0  3.0  5.0  3.0  4.0  5.0  2.0  2.0   
4      4  4.0  4.0  4.0  4.0  3.0  3.0  4.0  2.0  3.0  4.0  4.0  4.0  3.0   

   Q14  Q15  Q16  Q17  Q18  Q19  Q20  Q21  Q22  Q23  Q24  Q25  Q26 country  \
0  5.0  5.0  5.0  5.0  1.0  5.0  5.0  1.0  5.0  1.0  5.0  1.0  1.0     USA   
1  3.0  1.0  2.0  4.0  5.0  1.0  3.0  1.0  1.0  5.0  3.0  2.0  5.0     USA   
2  5.0  1.0  3.0  5.0  3.0  5.0  2.0  2.0  1.0  2.0  4.0  2.0  5.0     NLD   
3  4.0  4.0  2.0  4.0  5.0  4.0  3.0  3.0  4.0  3.0  4.0  4.0  2.0     USA   
4  5.0  5.0  2.0  4.0  1.0  4.0  2.0  4.0  2.0  3.0  4.0  4.0  4.0     ITA   

   introelapse  testelapse  surveyelapse  TIPI1  TIPI2  TIPI3  TIPI4

In [ ]:
corr_df = train_fill_nan.corr(method='pearson')
print(corr_df['nerdiness'])

index           0.002742
Q1              0.280343
Q2              0.260365
Q3              0.201872
Q4              0.261849
Q5              0.232420
Q6              0.306619
Q7              0.197176
Q8              0.224878
Q9              0.218848
Q10             0.182959
Q11             0.216964
Q12             0.242044
Q13             0.215772
Q14             0.229456
Q15             0.250658
Q16             0.175394
Q17             0.203361
Q18             0.180605
Q19             0.158807
Q20             0.212169
Q21             0.161261
Q22             0.160440
Q23             0.200013
Q24             0.224117
Q25             0.177759
Q26             0.236975
introelapse     0.008319
testelapse      0.005550
surveyelapse    0.011953
TIPI1          -0.095200
TIPI2           0.032783
TIPI3           0.046240
TIPI4           0.054102
TIPI5           0.048562
TIPI6           0.100961
TIPI7           0.048928
TIPI8           0.044987
TIPI9          -0.006465
TIPI10         -0.086178


In [ ]:
print(train_fill_nan)
print(train_fill_nan.columns)

       index   Q1   Q2   Q3   Q4   Q5   Q6   Q7   Q8   Q9  Q10  Q11  Q12  Q13  \
0          0  1.0  5.0  5.0  5.0  1.0  4.0  5.0  5.0  1.0  3.0  5.0  5.0  5.0   
1          1  4.0  4.0  4.0  4.0  4.0  5.0  4.0  4.0  3.0  3.0  1.0  4.0  5.0   
2          2  4.0  5.0  5.0  4.0  3.0  5.0  5.0  5.0  4.0  4.0  2.0  5.0  5.0   
3          3  4.0  4.0  4.0  2.0  4.0  3.0  3.0  5.0  3.0  4.0  5.0  2.0  2.0   
4          4  4.0  4.0  4.0  4.0  3.0  3.0  4.0  2.0  3.0  4.0  4.0  4.0  3.0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
14995  14995  2.0  5.0  4.0  3.0  3.0  4.0  4.0  4.0  3.0  4.0  1.0  4.0  4.0   
14996  14996  5.0  4.0  5.0  4.0  4.0  5.0  5.0  4.0  4.0  5.0  1.0  4.0  4.0   
14997  14997  4.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  4.0  5.0  4.0  5.0  5.0   
14998  14998  5.0  5.0  4.0  5.0  5.0  5.0  5.0  1.0  5.0  5.0  3.0  5.0  4.0   
14999  14999  5.0  4.0  2.0  5.0  2.0  2.0  4.0  2.0  4.0  4.0  4.0  4.0  3.0   

       Q14  Q15  Q16  Q17  

In [ ]:
test

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
0,0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,3.0,4.0,5.0,4.0,5.0,2.0,5.0,4.0,4.0,3.0,5.0,4.0,5.0,5.0,4.0,4.0,CHL,9,114,243,2.0,4.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0,1.0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,2.0,3,2.0,2.0,19,1.0,4.0,4.0,1.0,1.0,3.0,2.0
1,1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,4.0,4.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,5.0,4.0,5.0,USA,18,107,206,1.0,4.0,3.0,5.0,4.0,5.0,3.0,3.0,3.0,1.0,1,1,1,1,1,0,0,1,0,0,0,0,1,1,1,1,2.0,2,1.0,1.0,33,1.0,1.0,5.0,2.0,1.0,5.0,2.0
2,2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,1.0,5.0,5.0,4.0,4.0,3.0,4.0,5.0,2.0,3.0,5.0,4.0,5.0,5.0,1.0,5.0,IDN,4,87,191,3.0,3.0,4.0,3.0,4.0,4.0,5.0,3.0,2.0,1.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,1,1,1.0,3,2.0,2.0,13,1.0,4.0,5.0,2.0,1.0,3.0,2.0
3,3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,5.0,3.0,2.0,2.0,3.0,4.0,5.0,5.0,4.0,1.0,1.0,4.0,4.0,4.0,4.0,AUS,2409,155,258,1.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,1.0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,2.0,2,1.0,1.0,28,1.0,2.0,2.0,2.0,1.0,3.0,2.0
4,4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,5.0,5.0,4.0,1.0,5.0,2.0,3.0,5.0,5.0,3.0,3.0,PHL,24,163,289,3.0,3.0,3.0,5.0,5.0,4.0,3.0,4.0,3.0,4.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,2.0,3,2.0,2.0,15,1.0,4.0,5.0,2.0,1.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,5.0,4.0,5.0,4.0,3.0,5.0,5.0,4.0,4.0,5.0,4.0,3.0,4.0,4.0,5.0,3.0,USA,10,200,254,3.0,5.0,5.0,5.0,5.0,3.0,5.0,3.0,3.0,1.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,2.0,3,2.0,2.0,16,3.0,10.0,4.0,2.0,1.0,3.0,2.0
35448,35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,4.0,4.0,3.0,5.0,GBR,504,136,155,3.0,3.0,5.0,5.0,3.0,5.0,5.0,3.0,1.0,2.0,1,0,0,1,1,0,0,1,0,1,0,0,1,1,1,1,2.0,2,3.0,1.0,16,3.0,2.0,5.0,2.0,1.0,2.0,2.0
35449,35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,3.0,5.0,1.0,5.0,5.0,4.0,5.0,USA,27,121,191,1.0,5.0,4.0,3.0,4.0,3.0,3.0,5.0,2.0,1.0,1,1,1,1,1,0,0,0,0,1,1,0,1,1,1,1,3.0,3,1.0,1.0,31,1.0,1.0,5.0,1.0,1.0,2.0,2.0
35450,35450,5.0,5.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,4.0,5.0,1.0,1.0,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,CAN,3,58,105,3.0,5.0,5.0,1.0,5.0,5.0,5.0,1.0,5.0,1.0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,2.0,3,1.0,1.0,19,1.0,12.0,1.0,2.0,1.0,NaN,2.0


In [ ]:
print(test.shape)
print(test.isnull().sum())

(35452, 69)
index             0
Q1              119
Q2              147
Q3              130
Q4              125
Q5               90
Q6              132
Q7              142
Q8              108
Q9              104
Q10             220
Q11             119
Q12             149
Q13              96
Q14             102
Q15             107
Q16              85
Q17              79
Q18             147
Q19              97
Q20             119
Q21             123
Q22              82
Q23             114
Q24             125
Q25             108
Q26             108
country         450
introelapse       0
testelapse        0
surveyelapse      0
TIPI1           107
TIPI2           122
TIPI3           163
TIPI4           138
TIPI5           140
TIPI6           128
TIPI7           134
TIPI8           127
TIPI9           130
TIPI10          176
VCL1              0
VCL2              0
VCL3              0
VCL4              0
VCL5              0
VCL6              0
VCL7              0
VCL8              0
VCL9    

In [ ]:
test = test.fillna(0)

In [ ]:
train_fill_nan.drop(columns=['index','country'],inplace=True)
test.drop(columns=['index','country'], inplace=True)
print(test.shape)
print(train_fill_nan.shape)

(35452, 67)
(15000, 68)


In [ ]:
train_fill_nan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            15000 non-null  float64
 1   Q2            15000 non-null  float64
 2   Q3            15000 non-null  float64
 3   Q4            15000 non-null  float64
 4   Q5            15000 non-null  float64
 5   Q6            15000 non-null  float64
 6   Q7            15000 non-null  float64
 7   Q8            15000 non-null  float64
 8   Q9            15000 non-null  float64
 9   Q10           15000 non-null  float64
 10  Q11           15000 non-null  float64
 11  Q12           15000 non-null  float64
 12  Q13           15000 non-null  float64
 13  Q14           15000 non-null  float64
 14  Q15           15000 non-null  float64
 15  Q16           15000 non-null  float64
 16  Q17           15000 non-null  float64
 17  Q18           15000 non-null  float64
 18  Q19           15000 non-nu

In [ ]:
exp_clf = setup(data = train_fill_nan, target = 'nerdiness', normalize = False,
                train_size=0.99, fold=5 ,silent=True, data_split_stratify=True, use_gpu=True,
                preprocess=True, session_id=123)
add_metric('auc_proba', 'AUC_PROBA', roc_auc_score, greater_is_better=True, target="pred_proba")

,Description,Value
0,session_id,123
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(16606, 68)"
5,Missing Values,False
6,Numeric Features,50
7,Categorical Features,17
8,Ordinal Features,False
9,High Cardinality Features,False


Name                                                         AUC_PROBA
Display Name                                                 AUC_PROBA
Score Function              <function roc_auc_score at 0x7fc6d8712b90>
Scorer               make_scorer(roc_auc_score, needs_proba=True, e...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                 True
Multiclass                                                        True
Custom                                                            True
Name: auc_proba, dtype: object

In [ ]:
best_3 = compare_models(sort='auc_proba', n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC_PROBA,TT (Sec)
et,Extra Trees Classifier,0.7939,0.8884,0.7898,0.7965,0.7931,0.5878,0.5879,0.8884,3.226
rf,Random Forest Classifier,0.7895,0.8794,0.7899,0.7894,0.7896,0.5790,0.5791,0.8794,3.066
lightgbm,Light Gradient Boosting Machine,0.7521,0.8294,0.7639,0.7464,0.7550,0.5041,0.5043,0.8294,0.534
gbc,Gradient Boosting Classifier,0.7251,0.8009,0.7319,0.7222,0.7270,0.4502,0.4503,0.8009,4.752
ada,Ada Boost Classifier,0.7126,0.7868,0.7123,0.7128,0.7125,0.4251,0.4252,0.7868,1.064
lda,Linear Discriminant Analysis,0.7146,0.7824,0.7328,0.7072,0.7198,0.4293,0.4296,0.7824,0.236
lr,Logistic Regression,0.6866,0.7465,0.7085,0.6791,0.6934,0.3732,0.3737,0.7465,1.004
dt,Decision Tree Classifier,0.7086,0.7086,0.7095,0.7083,0.7089,0.4172,0.4173,0.7086,0.250
nb,Naive Bayes,0.5001,0.6979,0.0067,0.4859,0.0132,0.0002,-0.0001,0.6979,0.038
knn,K Neighbors Classifier,0.6094,0.6433,0.6220,0.6069,0.6143,0.2188,0.2189,0.6433,1.228


In [ ]:
blended_l = blend_models(best_3, fold = 3, optimize = 'auc_proba')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC_PROBA
Fold,,,,,,,,
0,0.7847,0.8755,0.7872,0.7832,0.7852,0.5693,0.5694,0.8755
1,0.7891,0.8755,0.7964,0.7849,0.7906,0.5781,0.5782,0.8755
2,0.7744,0.8650,0.7876,0.7674,0.7774,0.5488,0.5490,0.8650
Mean,0.7827,0.8720,0.7904,0.7785,0.7844,0.5654,0.5655,0.8720
Std,0.0061,0.0049,0.0042,0.0079,0.0054,0.0123,0.0122,0.0049


In [ ]:
blended_tuned = tune_model(blended_l, optimize='auc_proba')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC_PROBA
Fold,,,,,,,,
0,0.7895,0.8864,0.7932,0.7874,0.7903,0.5791,0.5791,0.8864
1,0.8050,0.8973,0.7987,0.8090,0.8038,0.6101,0.6101,0.8973
2,0.8060,0.8933,0.8102,0.8034,0.8068,0.6119,0.6119,0.8933
3,0.7871,0.8832,0.7993,0.7803,0.7897,0.5742,0.5744,0.8832
4,0.7913,0.8864,0.8047,0.7838,0.7941,0.5826,0.5828,0.8864
Mean,0.7958,0.8893,0.8012,0.7928,0.7969,0.5916,0.5917,0.8893
Std,0.0080,0.0052,0.0058,0.0113,0.0071,0.0161,0.0161,0.0052


In [ ]:
model = finalize_model(blended_tuned)

In [ ]:
pred = predict_model(model, data=test, raw_score=True) 

In [ ]:
pred['Score_1']

0        0.1298
1        0.7342
2        0.8616
3        0.6115
4        0.8976
          ...  
35447    0.9209
35448    0.6669
35449    0.8704
35450    0.0950
35451    0.6137
Name: Score_1, Length: 35452, dtype: float64

In [ ]:
submission = pd.read_csv('./drive/MyDrive/SW_AI/Dataset/sample_submission.csv')
submission['nerdiness'] = pred['Score_1']
submission

,index,nerdiness
0,0,0.1298
1,1,0.7342
2,2,0.8616
3,3,0.6115
4,4,0.8976
...,...,...
35447,35447,0.9209
35448,35448,0.6669
35449,35449,0.8704
35450,35450,0.0950


In [ ]:
submission.to_csv('./drive/MyDrive/SW_AI/Dataset/pycaret_cain_submission.csv', index = False)